# Internal Results

In [1]:
import os
from pathlib import Path
home = str(Path.home())
from scipy.stats.stats import pearsonr, spearmanr
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
import pandas as pd
# datadir = "/repositories/ai-DR/models/selectivity_D2D3/selectivity_models/models_C33_200/"
#datadir_dnn = "~/repositories/ai-DR/models/selectivity_D2D3/selectivity_models/models_C33_200/"
#datadir_dnn = "~/Shilab/sjwon3789/selectivity_models_C33_v19/models_C33_200/"
# datadir_dnn = "/home/wons2/repositories/ai-DR/models/selectivity_D2D3/selectivity_models/models_C33_200/"
#datadir = "/data/Shilab/sjwon3789/selectivity_models/models_C33_200/"
#datadir_dnn = "/data/Shilab/sjwon3789/selectivity_models/models_C33_200/"

datadir = "~/repositories/ai-DR/selectivity_models/models_C33_200/"
datadir_dnn = "~/repositories/ai-DR/selectivity_models/models_C33_200/"


def stats(y_true, y_pred):
    Rs = spearmanr(y_true, y_pred)[0]
    Rp = pearsonr(y_true, y_pred)[0]
    R2 = r2_score(y_true, y_pred)
    RMSE = sqrt(mean_squared_error(y_true, y_pred))
    return Rp, Rs, R2, RMSE


def predict_method(df, method):
    y_pred = list(df[method]) 
    y_true = list(df['exp_mean'])

    Rp, Rs, R2, RMSE = stats(y_true, y_pred)
    return Rp, Rs, R2, RMSE

In [2]:
import pandas as pd
import numpy as np

# Define a list of model directories
#model_directories = ["models__ratio_D2D3"]
model_directories = [ "models_D2", "models_D3", "models__ratio_D2D3"]

#model_directories = ["models_D3", "models__ratio_D2D3"]


# Create an empty list to store the results for each model
results = []

# Loop through each model directory
for model_dir in model_directories:
    # Create empty lists to store Rp, Rs, R2, and RMSE values for each method
    rp_values_consensus = []
    rs_values_consensus = []
    r2_values_consensus = []
    rmse_values_consensus = []

    rp_values_xgb = []
    rs_values_xgb = []
    r2_values_xgb = []
    rmse_values_xgb = []

    rp_values_rf = []
    rs_values_rf = []
    r2_values_rf = []
    rmse_values_rf = []

    rp_values_dnn = []
    rs_values_dnn = []
    r2_values_dnn = []
    rmse_values_dnn = []

    # Create empty lists to store max values for Rp, Rs, and R2, and min value for RMSE
    max_rp_values_consensus = []
    max_rs_values_consensus = []
    max_r2_values_consensus = []
    min_rmse_values_consensus = []

    max_rp_values_xgb = []
    max_rs_values_xgb = []
    max_r2_values_xgb = []
    min_rmse_values_xgb = []

    max_rp_values_rf = []
    max_rs_values_rf = []
    max_r2_values_rf = []
    min_rmse_values_rf = []

    max_rp_values_dnn = []
    max_rs_values_dnn = []
    max_r2_values_dnn = []
    min_rmse_values_dnn = []

    # Loop through different values of i
    for i in range(10):  # Assuming you have 10 values of i (0 to 9)
        df1 = pd.read_csv(datadir + f"{model_dir}/model_{i}/reg_xgb_0.15/pred_864.csv", sep="\t")
        df1 = df1[['compound', 'exp_mean', 'pred_864']]
        df2 = pd.read_csv(datadir + f"{model_dir}/model_{i}/reg_rf_0.15/pred_864.csv", sep="\t")
        df2 = df2[['compound', 'exp_mean', 'pred_864']]
        df3 = pd.read_csv(datadir_dnn + f"{model_dir}/model_{i}/reg_dnn_0.15/pred_864.csv", sep="\t")
        df3 = df3[['compound', 'exp_mean', 'pred_864']]

        # Rename the 'split_0' columns to 'xgb', 'rf', and 'dnn'
        df1.rename(columns={'pred_864': 'xgb'}, inplace=True)
        df2.rename(columns={'pred_864': 'rf'}, inplace=True)
        df3.rename(columns={'pred_864': 'dnn'}, inplace=True)

        # Merge the data frames based on the common columns 'compound' and 'exp_mean'
#         merged_df = pd.concat([df1, df2, df3], axis=1)

        merged_df = pd.merge(df1, df2, on=['compound', 'exp_mean'], how='outer')
        merged_df = pd.merge(merged_df, df3, on=['compound', 'exp_mean'], how='outer')
        merged_df["consensus"] = merged_df['xgb'] * 1/3 + merged_df["rf"] * 1/3 + merged_df["dnn"] * 1/3

        # Calculate statistical measures per each method and store them
        Rp, Rs, R2, RMSE = predict_method(merged_df, "consensus")
        rp_values_consensus.append(Rp)
        rs_values_consensus.append(Rs)
        r2_values_consensus.append(R2)
        rmse_values_consensus.append(RMSE)
        max_rp_values_consensus.append(np.max(rp_values_consensus))
        max_rs_values_consensus.append(np.max(rs_values_consensus))
        max_r2_values_consensus.append(np.max(r2_values_consensus))
        min_rmse_values_consensus.append(np.min(rmse_values_consensus))

        Rp, Rs, R2, RMSE = predict_method(merged_df, "xgb")
        rp_values_xgb.append(Rp)
        rs_values_xgb.append(Rs)
        r2_values_xgb.append(R2)
        rmse_values_xgb.append(RMSE)
        max_rp_values_xgb.append(np.max(rp_values_xgb))
        max_rs_values_xgb.append(np.max(rs_values_xgb))
        max_r2_values_xgb.append(np.max(r2_values_xgb))
        min_rmse_values_xgb.append(np.min(rmse_values_xgb))

        Rp, Rs, R2, RMSE = predict_method(merged_df, "rf")
        rp_values_rf.append(Rp)
        rs_values_rf.append(Rs)
        r2_values_rf.append(R2)
        rmse_values_rf.append(RMSE)
        max_rp_values_rf.append(np.max(rp_values_rf))
        max_rs_values_rf.append(np.max(rs_values_rf))
        max_r2_values_rf.append(np.max(r2_values_rf))
        min_rmse_values_rf.append(np.min(rmse_values_rf))

        Rp, Rs, R2, RMSE = predict_method(merged_df, "dnn")
        rp_values_dnn.append(Rp)
        rs_values_dnn.append(Rs)
        r2_values_dnn.append(R2)
        rmse_values_dnn.append(RMSE)
        max_rp_values_dnn.append(np.max(rp_values_dnn))
        max_rs_values_dnn.append(np.max(rs_values_dnn))
        max_r2_values_dnn.append(np.max(r2_values_dnn))
        min_rmse_values_dnn.append(np.min(rmse_values_dnn))

    # Create a data frame to store the results for the current model
    model_results_df = pd.DataFrame({
        'Rp_mean': [f"{np.mean(rp_values_xgb):.2f} ({np.std(rp_values_xgb):.2f})",
                    f"{np.mean(rp_values_rf):.2f} ({np.std(rp_values_rf):.2f})",
                    f"{np.mean(rp_values_dnn):.2f} ({np.std(rp_values_dnn):.2f})",
                    f"{np.mean(rp_values_consensus):.2f} ({np.std(rp_values_consensus):.2f})"],
        'Rp_max': [f"{np.max(max_rp_values_xgb):.2f}",
                   f"{np.max(max_rp_values_rf):.2f}",
                   f"{np.max(max_rp_values_dnn):.2f}",
                   f"{np.max(max_rp_values_consensus):.2f}"],
        'Rs_mean': [f"{np.mean(rs_values_xgb):.2f} ({np.std(rs_values_xgb):.2f})",
                    f"{np.mean(rs_values_rf):.2f} ({np.std(rs_values_rf):.2f})",
                    f"{np.mean(rs_values_dnn):.2f} ({np.std(rs_values_dnn):.2f})",
                    f"{np.mean(rs_values_consensus):.2f} ({np.std(rs_values_consensus):.2f})"],
        'Rs_max': [f"{np.max(max_rs_values_xgb):.2f}",
                   f"{np.max(max_rs_values_rf):.2f}",
                   f"{np.max(max_rs_values_dnn):.2f}",
                   f"{np.max(max_rs_values_consensus):.2f}"],
        'R2_mean': [f"{np.mean(r2_values_xgb):.2f} ({np.std(r2_values_xgb):.2f})",
                    f"{np.mean(r2_values_rf):.2f} ({np.std(r2_values_rf):.2f})",
                    f"{np.mean(r2_values_dnn):.2f} ({np.std(r2_values_dnn):.2f})",
                    f"{np.mean(r2_values_consensus):.2f} ({np.std(r2_values_consensus):.2f})"],
        'R2_max': [f"{np.max(max_r2_values_xgb):.2f}",
                   f"{np.max(max_r2_values_rf):.2f}",
                   f"{np.max(max_r2_values_dnn):.2f}",
                   f"{np.max(max_r2_values_consensus):.2f}"],
        'RMSE_mean': [f"{np.mean(rmse_values_xgb):.2f} ({np.std(rmse_values_xgb):.2f})",
                      f"{np.mean(rmse_values_rf):.2f} ({np.std(rmse_values_rf):.2f})",
                      f"{np.mean(rmse_values_dnn):.2f} ({np.std(rmse_values_dnn):.2f})",
                      f"{np.mean(rmse_values_consensus):.2f} ({np.std(rmse_values_consensus):.2f})"],
        'RMSE_min': [f"{np.min(min_rmse_values_xgb):.2f}",
                     f"{np.min(min_rmse_values_rf):.2f}",
                     f"{np.min(min_rmse_values_dnn):.2f}",
                     f"{np.min(min_rmse_values_consensus):.2f}"]
    }, index=['xgb', 'rf', 'dnn', 'consensus'])

    # Append the results for the current model to the overall results list
    results.append((model_dir, model_results_df))

# Create a data frame to store the results for all models
final_results_df = pd.concat([result[1] for result in results], keys=[result[0] for result in results])
final_results_df.to_csv("output_dir/consensus_regression_internal.csv")
final_results_df

Rp_mean Rp_max      Rs_mean Rs_max  \
models_D2          xgb        0.82 (0.01)   0.84  0.82 (0.01)   0.84   
                   rf         0.82 (0.01)   0.84  0.81 (0.01)   0.84   
                   dnn        0.83 (0.01)   0.85  0.83 (0.01)   0.85   
                   consensus  0.85 (0.01)   0.86  0.84 (0.01)   0.87   
models_D3          xgb        0.82 (0.02)   0.84  0.82 (0.02)   0.84   
                   rf         0.82 (0.03)   0.84  0.82 (0.02)   0.84   
                   dnn        0.82 (0.02)   0.86  0.82 (0.02)   0.86   
                   consensus  0.83 (0.02)   0.86  0.83 (0.02)   0.86   
models__ratio_D2D3 xgb        0.86 (0.02)   0.89  0.86 (0.02)   0.90   
                   rf         0.86 (0.02)   0.89  0.85 (0.03)   0.90   
                   dnn        0.86 (0.01)   0.87  0.86 (0.01)   0.89   
                   consensus  0.87 (0.01)   0.89  0.87 (0.02)   0.91   

                                  R2_mean R2_max    RMSE_mean RMSE_min  
models_D2          xgb        0.68 (0.02)   0.70  0.63 (0.02)     0.60  
                   rf         0.66 (0.02)   0.68  0.65 (0.02)     0.62  
                   dnn        0.69 (0.02)   0.72  0.62 (0.02)     0.58  
                   consensus  0.71 (0.02)   0.73  0.60 (0.02)     0.57  
models_D3          xgb        0.67 (0.04)   0.71  0.68 (0.03)     0.64  
                   rf         0.66 (0.04)   0.70  0.69 (0.04)     0.64  
                   dnn        0.66 (0.04)   0.74  0.69 (0.03)     0.64  
                   consensus  0.69 (0.04)   0.74  0.66 (0.03)     0.62  
models__ratio_D2D3 xgb        0.74 (0.03)   0.78  0.53 (0.03)     0.50  
                   rf         0.73 (0.04)   0.78  0.54 (0.04)     0.50  
                   dnn        0.73 (0.02)   0.76  0.54 (0.02)     0.50  
                   consensus  0.75 (0.02)   0.79  0.52 (0.03)     0.48

# External Results

In [3]:
import pandas as pd
import numpy as np

# Define a list of model directories
model_directories = ["models_D2", "models_D3", "models__ratio_D2D3"]

# Create an empty list to store the results for each model
results = []

# Loop through each model directory
for model_dir in model_directories:
    # Create empty lists to store Rp, Rs, R2, and RMSE values for each method
    rp_values_consensus = []
    rs_values_consensus = []
    r2_values_consensus = []
    rmse_values_consensus = []

    rp_values_xgb = []
    rs_values_xgb = []
    r2_values_xgb = []
    rmse_values_xgb = []

    rp_values_rf = []
    rs_values_rf = []
    r2_values_rf = []
    rmse_values_rf = []

    rp_values_dnn = []
    rs_values_dnn = []
    r2_values_dnn = []
    rmse_values_dnn = []

    # Create empty lists to store max values for Rp, Rs, and R2, and min value for RMSE
    max_rp_values_consensus = []
    max_rs_values_consensus = []
    max_r2_values_consensus = []
    min_rmse_values_consensus = []

    max_rp_values_xgb = []
    max_rs_values_xgb = []
    max_r2_values_xgb = []
    min_rmse_values_xgb = []

    max_rp_values_rf = []
    max_rs_values_rf = []
    max_r2_values_rf = []
    min_rmse_values_rf = []

    max_rp_values_dnn = []
    max_rs_values_dnn = []
    max_r2_values_dnn = []
    min_rmse_values_dnn = []

    # Loop through different values of i
    for i in range(10):  # Assuming you have 10 values of i (0 to 9)
        df1 = pd.read_csv(datadir + f"{model_dir}/model_{i}/pred_reg_xgb_0.00/pred_splits_val{i}", sep="\t")
        df1 = df1[['compound', 'exp_mean', 'split_0']]
        df2 = pd.read_csv(datadir + f"{model_dir}/model_{i}/pred_reg_rf_0.00/pred_splits_val{i}", sep="\t")
        df2 = df2[['compound', 'exp_mean', 'split_0']]
        df3 = pd.read_csv(datadir_dnn + f"{model_dir}/model_{i}/pred_reg_dnn_0.00/pred_splits", sep="\t")
        df3 = df3[['compound', 'exp_mean', 'split_0']]

        # Rename the 'split_0' columns to 'xgb', 'rf', and 'dnn'
        df1.rename(columns={'split_0': 'xgb'}, inplace=True)
        df2.rename(columns={'split_0': 'rf'}, inplace=True)
        df3.rename(columns={'split_0': 'dnn'}, inplace=True)

        # Merge the data frames based on the common columns 'compound' and 'exp_mean'
        merged_df = pd.merge(df1, df2, on=['compound', 'exp_mean'], how='outer')
        merged_df = pd.merge(merged_df, df3, on=['compound', 'exp_mean'], how='outer')
        merged_df["consensus"] = merged_df['xgb'] * 1/3 + merged_df["rf"] * 1/3 + merged_df["dnn"] * 1/3

        # Calculate statistical measures per each method and store them
        Rp, Rs, R2, RMSE = predict_method(merged_df, "consensus")
        rp_values_consensus.append(Rp)
        rs_values_consensus.append(Rs)
        r2_values_consensus.append(R2)
        rmse_values_consensus.append(RMSE)
        max_rp_values_consensus.append(np.max(rp_values_consensus))
        max_rs_values_consensus.append(np.max(rs_values_consensus))
        max_r2_values_consensus.append(np.max(r2_values_consensus))
        min_rmse_values_consensus.append(np.min(rmse_values_consensus))

        Rp, Rs, R2, RMSE = predict_method(merged_df, "xgb")
        rp_values_xgb.append(Rp)
        rs_values_xgb.append(Rs)
        r2_values_xgb.append(R2)
        rmse_values_xgb.append(RMSE)
        max_rp_values_xgb.append(np.max(rp_values_xgb))
        max_rs_values_xgb.append(np.max(rs_values_xgb))
        max_r2_values_xgb.append(np.max(r2_values_xgb))
        min_rmse_values_xgb.append(np.min(rmse_values_xgb))

        Rp, Rs, R2, RMSE = predict_method(merged_df, "rf")
        rp_values_rf.append(Rp)
        rs_values_rf.append(Rs)
        r2_values_rf.append(R2)
        rmse_values_rf.append(RMSE)
        max_rp_values_rf.append(np.max(rp_values_rf))
        max_rs_values_rf.append(np.max(rs_values_rf))
        max_r2_values_rf.append(np.max(r2_values_rf))
        min_rmse_values_rf.append(np.min(rmse_values_rf))

        Rp, Rs, R2, RMSE = predict_method(merged_df, "dnn")
        rp_values_dnn.append(Rp)
        rs_values_dnn.append(Rs)
        r2_values_dnn.append(R2)
        rmse_values_dnn.append(RMSE)
        max_rp_values_dnn.append(np.max(rp_values_dnn))
        max_rs_values_dnn.append(np.max(rs_values_dnn))
        max_r2_values_dnn.append(np.max(r2_values_dnn))
        min_rmse_values_dnn.append(np.min(rmse_values_dnn))

    # Create a data frame to store the results for the current model
    model_results_df = pd.DataFrame({
        'Rp_mean': [f"{np.mean(rp_values_xgb):.2f} ({np.std(rp_values_xgb):.2f})",
                    f"{np.mean(rp_values_rf):.2f} ({np.std(rp_values_rf):.2f})",
                    f"{np.mean(rp_values_dnn):.2f} ({np.std(rp_values_dnn):.2f})",
                    f"{np.mean(rp_values_consensus):.2f} ({np.std(rp_values_consensus):.2f})"],
        'Rp_max': [f"{np.max(max_rp_values_xgb):.2f}",
                   f"{np.max(max_rp_values_rf):.2f}",
                   f"{np.max(max_rp_values_dnn):.2f}",
                   f"{np.max(max_rp_values_consensus):.2f}"],
        'Rs_mean': [f"{np.mean(rs_values_xgb):.2f} ({np.std(rs_values_xgb):.2f})",
                    f"{np.mean(rs_values_rf):.2f} ({np.std(rs_values_rf):.2f})",
                    f"{np.mean(rs_values_dnn):.2f} ({np.std(rs_values_dnn):.2f})",
                    f"{np.mean(rs_values_consensus):.2f} ({np.std(rs_values_consensus):.2f})"],
        'Rs_max': [f"{np.max(max_rs_values_xgb):.2f}",
                   f"{np.max(max_rs_values_rf):.2f}",
                   f"{np.max(max_rs_values_dnn):.2f}",
                   f"{np.max(max_rs_values_consensus):.2f}"],
        'R2_mean': [f"{np.mean(r2_values_xgb):.2f} ({np.std(r2_values_xgb):.2f})",
                    f"{np.mean(r2_values_rf):.2f} ({np.std(r2_values_rf):.2f})",
                    f"{np.mean(r2_values_dnn):.2f} ({np.std(r2_values_dnn):.2f})",
                    f"{np.mean(r2_values_consensus):.2f} ({np.std(r2_values_consensus):.2f})"],
        'R2_max': [f"{np.max(max_r2_values_xgb):.2f}",
                   f"{np.max(max_r2_values_rf):.2f}",
                   f"{np.max(max_r2_values_dnn):.2f}",
                   f"{np.max(max_r2_values_consensus):.2f}"],
        'RMSE_mean': [f"{np.mean(rmse_values_xgb):.2f} ({np.std(rmse_values_xgb):.2f})",
                      f"{np.mean(rmse_values_rf):.2f} ({np.std(rmse_values_rf):.2f})",
                      f"{np.mean(rmse_values_dnn):.2f} ({np.std(rmse_values_dnn):.2f})",
                      f"{np.mean(rmse_values_consensus):.2f} ({np.std(rmse_values_consensus):.2f})"],
        'RMSE_min': [f"{np.min(min_rmse_values_xgb):.2f}",
                     f"{np.min(min_rmse_values_rf):.2f}",
                     f"{np.min(min_rmse_values_dnn):.2f}",
                     f"{np.min(min_rmse_values_consensus):.2f}"]
    }, index=['xgb', 'rf', 'dnn', 'consensus'])

    # Append the results for the current model to the overall results list
    results.append((model_dir, model_results_df))

# Create a data frame to store the results for all models
final_results_df = pd.concat([result[1] for result in results], keys=[result[0] for result in results])
final_results_df.to_csv("output_dir/consensus_regression_external.csv")
final_results_df

Rp_mean Rp_max      Rs_mean Rs_max  \
models_D2          xgb        0.82 (0.03)   0.88  0.82 (0.02)   0.85   
                   rf         0.82 (0.03)   0.88  0.81 (0.02)   0.85   
                   dnn        0.83 (0.03)   0.87  0.82 (0.02)   0.85   
                   consensus  0.84 (0.03)   0.89  0.84 (0.02)   0.86   
models_D3          xgb        0.85 (0.02)   0.87  0.84 (0.02)   0.87   
                   rf         0.85 (0.02)   0.88  0.84 (0.02)   0.87   
                   dnn        0.85 (0.02)   0.88  0.84 (0.02)   0.88   
                   consensus  0.86 (0.02)   0.89  0.86 (0.02)   0.89   
models__ratio_D2D3 xgb        0.86 (0.03)   0.90  0.85 (0.03)   0.90   
                   rf         0.86 (0.03)   0.89  0.85 (0.03)   0.89   
                   dnn        0.86 (0.02)   0.89  0.86 (0.02)   0.89   
                   consensus  0.87 (0.03)   0.90  0.86 (0.02)   0.90   

                                  R2_mean R2_max    RMSE_mean RMSE_min  
models_D2          xgb        0.67 (0.04)   0.75  0.60 (0.03)     0.55  
                   rf         0.66 (0.05)   0.76  0.61 (0.04)     0.53  
                   dnn        0.68 (0.05)   0.75  0.60 (0.04)     0.52  
                   consensus  0.70 (0.04)   0.77  0.58 (0.04)     0.49  
models_D3          xgb        0.72 (0.03)   0.76  0.65 (0.04)     0.61  
                   rf         0.71 (0.03)   0.76  0.66 (0.04)     0.61  
                   dnn        0.71 (0.04)   0.77  0.66 (0.05)     0.59  
                   consensus  0.74 (0.03)   0.78  0.63 (0.04)     0.57  
models__ratio_D2D3 xgb        0.74 (0.05)   0.80  0.52 (0.05)     0.44  
                   rf         0.74 (0.05)   0.79  0.52 (0.05)     0.46  
                   dnn        0.74 (0.04)   0.78  0.52 (0.04)     0.47  
                   consensus  0.76 (0.04)   0.80  0.50 (0.05)     0.44